# Let's code a Neural Network in plain NumPy
---

***Author: Piotr Skalski***

Using high-level frameworks like Keras, TensorFlow or PyTorch allows us to build very complex models quickly. However, it is worth taking the time to look inside and understand underlying concepts. Not so long ago I published an article in which I tried to explain - in a simple way - how neural nets work. However, it was  highly theoretical post, dedicated primarily to math, which is the source of NN superpower. From the very beginning I was planning to follow up this topic in a more practical way. This time we will try to make use of our knowledge and build a fully operational neural network using only NumPy. Finally, we will also use it to solve simple classification problems and compare its performance with the model built with Keras.

### Imports

In [2]:
# set the file name (required)
__file__ = 'Numpy deep neural network.ipynb'

import numpy as np
from pprint import pprint
from IPython.display import Image

from tqdm import tqdm

import ipytest.magics
import pytest

### Network architecture

![Network architecture](./supporting_visualizations/nn_architecture.png)

<b>Figure 1.</b> Example of dense neural network architecture

### First things first

Before we start programming, let's stop for a moment and prepare a basic roadmap. Our goal is to create a program capable of creating a densely connected neural network with the specified architecture (number and size of layers and appropriate activation function). An example of such a network is presented in Figure 1. Above all, we must be able to train our network and make predictions using it.

![Roadmap](./supporting_visualizations/blueprint.gif)

<b>Figure 2.</b> Neural network blueprint

Diagram above shows what operations will have to be performed during the training of our neural network. It also shows how many parameters we will have to update and read at different stages of a single iteration. Building the right data structure and skillfully managing its state is the most difficult part of our task.

### Initiation of neural network layers

![Parameters sizes](./supporting_visualizations/params_sizes.png)

<b>Figure 3.</b> Dimensions of weight matrix W and bias vector b for layer l.

Let's start with by initiating weight matrix W and bias vector b for each layer. In Figure 3 I have prepared a small cheatsheet, which will help us to asign the appropriate dimensions for these coefficients. Superscript [l] denotes the index of the current layer (counted from 1). I assumed that the information describing the NN architecture will be delivered to our program in the form of list similar to the one presented on Snippet 1. Each item in the list is a dictionary describing the basic parameters of a single network layer: input_dim - the size of the signal vector supplied as an input for the layer, output_dim - the size of the activation vector obtained at the output of the layer and activation - the activation function to be used inside the layer.

In [53]:
NN_ARCHITECTURE = [
    {"input_dim": 2, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 200, "activation": "relu"},
    {"input_dim": 200, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 1, "activation": "sigmoid"},
]

### Activation functions

![Activations](./supporting_visualizations/activations.png)

In [4]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

### Initiation of parameter values for each layer

In [50]:
def init_layers(nn_architecture, seed = 99):
    # random seed initiation
    np.random.seed(seed)
    # number of layers in our neural network
    number_of_layers = len(nn_architecture)
    # parameters storage initiation
    params_values = {}
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        
        # extracting the number of units in layers
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        # initiating the values of the W matrix
        # and vector b for subsequent layers
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    return params_values

In [6]:
%%run_pytest[clean] -qq
# TEST PARAMETERS SHAPES

params_values = init_layers(NN_ARCHITECTURE)

def test_first_layer_W_shape():
    assert params_values["W1"].shape == (NN_ARCHITECTURE[0]["output_dim"], NN_ARCHITECTURE[0]["input_dim"])
def test_first_layer_b_shape():
    assert params_values["b1"].shape == (NN_ARCHITECTURE[0]["output_dim"], 1)
def test_first_layer_W_shape():
    assert params_values["W2"].shape == (NN_ARCHITECTURE[1]["output_dim"], NN_ARCHITECTURE[1]["input_dim"])
def test_first_layer_b_shape():
    assert params_values["b2"].shape == (NN_ARCHITECTURE[1]["output_dim"], 1)

..                                                                         [100%]


### Single layer forward propagation step

![Single unit](./supporting_visualizations/single_unit.png)

In [51]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    # calculation of the input value for the activation function
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    # selection of activation function
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    # return of calculated activation A and the intermediate Z matrix
    return activation_func(Z_curr), Z_curr

![Matrix sizes](./supporting_visualizations/matrix_sizes.png)

![Matrix sizes 2](./supporting_visualizations/matrix_sizes_2.png)

In [8]:
%%run_pytest[clean] -qq
# TEST OUTPUT FOR SINGLE LAYER FORWARD STEP

np.random.seed(2)
A_prev = np.random.randn(3,2)
W_curr = np.random.randn(1,3)
b_curr = np.random.randn(1,1)

A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu")

def test_relu_Z_shape():
    assert Z_curr.shape == (1,2)
def test_relu_A_shape():
    assert A_curr.shape == (1,2)
def test_relu_Z_value():
    assert np.allclose(Z_curr, np.array([[ 3.43896131, -2.08938436]]))
def test_relu_A_value():
    assert np.allclose(A_curr, np.array([[3.43896131, 0.        ]]))

....                                                                       [100%]


### Full forward propagation

In [52]:
def full_forward_propagation(X, params_values, nn_architecture):
    # creating a temporary memory to store the information needed for a backward step
    memory = {}
    # X vector is the activation for layer 0 
    A_curr = X
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        # transfer the activation from the previous iteration
        A_prev = A_curr
        
        # extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
        # extraction of W for the current layer
        W_curr = params_values["W" + str(layer_idx)]
        # extraction of b for the current layer
        b_curr = params_values["b" + str(layer_idx)]
        # calculation of activation for the current layer
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        # saving calculated values in the memory
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    # return of prediction vector and a dictionary containing intermediate values
    return A_curr, memory

In [10]:
np.random.seed(6)
X = np.random.randn(5, 4)
W1 = np.random.randn(4, 5)
b1 = np.random.randn(4, 1)
W2 = np.random.randn(3, 4)
b2 = np.random.randn(3, 1)
W3 = np.random.randn(1, 3)
b3 = np.random.randn(1, 1)

params_values = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
                "b3": b3}
pprint(X)

array([[-0.31178367,  0.72900392,  0.21782079, -0.8990918 ],
       [-2.48678065,  0.91325152,  1.12706373, -1.51409323],
       [ 1.63929108, -0.4298936 ,  2.63128056,  0.60182225],
       [-0.33588161,  1.23773784,  0.11112817,  0.12915125],
       [ 0.07612761, -0.15512816,  0.63422534,  0.810655  ]])


In [11]:
nn_architecture = [
    {"input_dim": 4, "output_dim": 5, "activation": "relu"},
    {"input_dim": 4, "output_dim": 3, "activation": "relu"},
    {"input_dim": 3, "output_dim": 1, "activation": "sigmoid"},
]

In [12]:
A, memory = full_forward_propagation(X, params_values, nn_architecture)

In [13]:
pprint(memory)

{'A0': array([[-0.31178367,  0.72900392,  0.21782079, -0.8990918 ],
       [-2.48678065,  0.91325152,  1.12706373, -1.51409323],
       [ 1.63929108, -0.4298936 ,  2.63128056,  0.60182225],
       [-0.33588161,  1.23773784,  0.11112817,  0.12915125],
       [ 0.07612761, -0.15512816,  0.63422534,  0.810655  ]]),
 'A1': array([[0.        , 3.18040136, 0.4074501 , 0.        ],
       [0.        , 0.        , 3.18141623, 0.        ],
       [4.18500916, 0.        , 0.        , 2.72141638],
       [5.05850802, 0.        , 0.        , 3.82321852]]),
 'A2': array([[ 2.2644603 ,  1.09971298,  0.        ,  1.54036335],
       [ 6.33722569,  0.        ,  0.        ,  4.48582383],
       [10.37508342,  0.        ,  1.63635185,  8.17870169]]),
 'Z1': array([[-5.23825714,  3.18040136,  0.4074501 , -1.88612721],
       [-2.77358234, -0.56177316,  3.18141623, -0.99209432],
       [ 4.18500916, -1.78006909, -0.14502619,  2.72141638],
       [ 5.05850802, -1.25674082, -3.54566654,  3.82321852]]),
 'Z2

### Calculating the value of the cost function

In [14]:
def get_cost_value(Y_hat, Y):
    # number of examples
    m = Y_hat.shape[1]
    # calculation of the cost according to the formula
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

In [15]:
Y = np.asarray([[1, 1, 1]])
Y_hat = np.array([[.8, .9, 0.4]])
get_cost_value(Y_hat, Y)

array(0.4149316)

### Single layer backward propagation step

In [16]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    # number of examples
    m = A_prev.shape[1]
    
    # selection of activation function
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
        
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    
    # derivative of the matrix W
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    # derivative of the vector b
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    # derivative of the matrix A_prev
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

In [17]:
np.random.seed(2)
dA_curr = np.random.randn(1, 2)
A_prev = np.random.randn(3, 2)
W_curr = np.random.randn(1, 3)
b_curr = np.random.randn(1, 1)
Z_curr = np.random.randn(1, 2)

single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu")

(array([[ 0.44090989,  0.        ],
        [ 0.37883606,  0.        ],
        [-0.2298228 ,  0.        ]]),
 array([[ 0.44513824,  0.37371418, -0.10478989]]),
 array([[-0.20837892]]))

### Full backward propagation

$$\boldsymbol{dW}^{[l]} = \frac{\partial L }{\partial \boldsymbol{W}^{[l]}} = \frac{1}{m} \boldsymbol{dZ}^{[l]} \boldsymbol{A}^{[l-1] T}$$

$$\boldsymbol{db}^{[l]} = \frac{\partial L }{\partial \boldsymbol{b}^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} \boldsymbol{dZ}^{[l](i)}$$

$$\boldsymbol{dA}^{[l-1]} = \frac{\partial L }{\partial \boldsymbol{A}^{[l-1]}} = \boldsymbol{W}^{[l] T} \boldsymbol{dZ}^{[l]}$$

$$\boldsymbol{dZ}^{[l]} = \boldsymbol{dA}^{[l]} * g'(\boldsymbol{Z}^{[l]})$$

In [18]:
def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    
    # number of examples
    m = Y.shape[1]
    # a hack ensuring the same shape of the prediction vector and labels vector
    Y = Y.reshape(Y_hat.shape)
    
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        # we number network layers from 1
        layer_idx_curr = layer_idx_prev + 1
        # extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

In [19]:
np.random.seed(3)
Y_hat = np.random.randn(1, 2)
Y = np.array([[1, 0]])

A0 = np.random.randn(4, 2)
W1 = np.random.randn(3, 4)
b1 = np.random.randn(3, 1)
Z1 = np.random.randn(3, 2)

A1 = np.random.randn(3, 2)
W2 = np.random.randn(1, 3)
b2 = np.random.randn(1, 1)
Z2 = np.random.randn(1, 2)

params_values = {}
params_values["W1"] = W1
params_values["b1"] = b1
params_values["W2"] = W2
params_values["b2"] = b2

memory = {}
memory["A0"] = A0
memory["Z1"] = Z1
memory["A1"] = A1
memory["Z2"] = Z2

nn_architecture = [
    {"input_dim": 4, "output_dim": 3, "activation": "relu"},
    {"input_dim": 3, "output_dim": 1, "activation": "sigmoid"},
]

full_backward_propagation(Y_hat, Y, memory,  params_values, nn_architecture)

{'dW1': array([[0.41010002, 0.07807203, 0.13798444, 0.10502167],
        [0.        , 0.        , 0.        , 0.        ],
        [0.05283652, 0.01005865, 0.01777766, 0.0135308 ]]),
 'dW2': array([[-0.39202432, -0.13325855, -0.04601089]]),
 'db1': array([[-0.22007063],
        [ 0.        ],
        [-0.02835349]]),
 'db2': array([[0.15187861]])}

### Updating parameter values

In [20]:
def update(params_values, grads_values, nn_architecture, learning_rate):

    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values;

In [21]:
np.random.seed(2)
W1 = np.random.randn(3, 4)
b1 = np.random.randn(3, 1)
W2 = np.random.randn(1, 3)
b2 = np.random.randn(1, 1)
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2}
np.random.seed(3)
dW1 = np.random.randn(3, 4)
db1 = np.random.randn(3, 1)
dW2 = np.random.randn(1, 3)
db2 = np.random.randn(1, 1)
grads = {"dW1": dW1,
         "db1": db1,
         "dW2": dW2,
         "db2": db2}

nn_architecture = [
    {"input_dim": 4, "output_dim": 3, "activation": "relu"},
    {"input_dim": 3, "output_dim": 1, "activation": "sigmoid"},
]

update(parameters, grads, nn_architecture, 0.1)

{'W1': array([[-0.59562069, -0.09991781, -2.14584584,  1.82662008],
        [-1.76569676, -0.80627147,  0.51115557, -1.18258802],
        [-1.0535704 , -0.86128581,  0.68284052,  2.20374577]]),
 'W2': array([[-0.55569196,  0.0354055 ,  1.32964895]]),
 'b1': array([[-0.04659241],
        [-1.28888275],
        [ 0.53405496]]),
 'b2': array([[-0.84610769]])}

In [41]:
def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    
    for i in range(epochs):
#     for i in tqdm(range(epochs)):
#         print(params_values["W1"][0])
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        if(i % 200 == 0):
            print("Iteration: {} - cost: {}".format(cost, i))
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
#         print(grads_values["dW2"][0])
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values

In [42]:
# number of samples in the data set
N_SAMPLES = 1000
# ratio between training and test sets
TEST_SIZE = 0.1
# number of iterations of the model
N_EPOCHS = 50

In [43]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

In [44]:
X, y = make_circles(n_samples=N_SAMPLES, factor=.3, noise=.10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [49]:
params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 5000, 0.01)

Iteration: 0.6938129619229754 - cost: 0
Iteration: 0.6895829493236217 - cost: 200
Iteration: 0.6858955922538015 - cost: 400
Iteration: 0.6801361868834316 - cost: 600
Iteration: 0.6702499234379141 - cost: 800
Iteration: 0.6493653695640564 - cost: 1000
Iteration: 0.5967080898232197 - cost: 1200
Iteration: 0.4389623136346585 - cost: 1400
Iteration: 0.16235153585245227 - cost: 1600
Iteration: 0.052629948923844705 - cost: 1800
Iteration: 0.025107234623227174 - cost: 2000
Iteration: 0.015233370185100945 - cost: 2200
Iteration: 0.010561400874414954 - cost: 2400
Iteration: 0.007927754253277684 - cost: 2600
Iteration: 0.006273385746634368 - cost: 2800
Iteration: 0.0051496810519635625 - cost: 3000
Iteration: 0.004342984058184206 - cost: 3200
Iteration: 0.003740084696233018 - cost: 3400
Iteration: 0.003273886623793503 - cost: 3600
Iteration: 0.002903615822674675 - cost: 3800
Iteration: 0.002603240310480528 - cost: 4000
Iteration: 0.0023553226784523156 - cost: 4200
Iteration: 0.002148204576107743 

In [543]:
y_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)

In [544]:
y_test

array([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1])

In [545]:
y_hat[y_hat > 0.5] = 1
y_hat[y_hat <= 0.5] = 0
y_hat

array([[0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 1.]])

In [546]:
from sklearn.metrics import accuracy_score

In [549]:
y_test.shape

(100,)

In [553]:
y_hat.reshape((y_hat.shape[1], 1)).shape

(100, 1)

In [554]:
accuracy_score(y_test, y_hat.reshape((y_hat.shape[1], 1)))

1.0